# **Auxilliary**

In [45]:
# Library imports
import sys
import pandas as pd

from numpy.random import seed

from dataset import dataset_generator
from bot.bot import Bot

path = "binary-options-research/"
if path not in sys.path:
    sys.path.append(path)

seed(42)

%load_ext autoreload
%autoreload 2

# **Data Generation**

In [12]:
# Generate dataset
sys.stdout = open("stdout", 'w')
dataset_generator.make(["BTCUSDT"], "1m")
sys.stdout.close()

/Users/mrspooks/Desktop/Capstone/binary-options-research/dataset/data_utils.py:16: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  candles[["open_time", "close_time"]] = candles[["open_time", "close_time"]].apply(
/Users/mrspooks/Desktop/Capstone/binary-options-research/dataset/data_utils.py:16: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  candles[["open_time", "close_time"]] = candles[["open_time", "close_time"]].apply(
/Users/mrspooks/Desktop/Capstone/binary-options-research/datas

# **Data Extraction**

In [28]:
# Extract data
candles = pd.read_csv(f"dataset/BTCUSDT-1m-2020_2024.csv", index_col=0)
candles_training = candles[(candles['open_time'] >= '2020-01-01') & (candles['open_time'] < '2024-01-01')].reset_index(drop=True)

In [56]:
sys.stdout = open("stdout", 'w')
print(f'Running data extraction \n')

bot = Bot(candles)
in_position = 0
timespan = 0
wins = 0
total_trades = 0

for i in range(1000, len(candles_training)):
    if not in_position:
        in_position, entry, position = bot._process_candle(candles_training.iloc[i])
        timespan = 0
    if in_position:
        if timespan != 10:
            timespan += 1
        else:
            in_position = 0
            print(f'Closed position on {candles_training.iloc[i]["close"]} at {candles_training.iloc[i]["open_time"]}')

            if position == "long":
                if entry < candles_training.iloc[i]["close"]:
                    wins += 1
                    print ("Trade won")
                else:
                    print ("Trade lost")
            else:
                if entry > candles_training.iloc[i]["close"]:
                    wins += 1
                    print ("Trade won")
                else:
                    print ("Trade lost")
            
            total_trades += 1
            candles_training = candles_training[i:]
            print(f"WR: {round(wins / total_trades * 100, 2)}% \n")


sys.stdout.close()

IndexError: single positional indexer is out-of-bounds